# Cardiovascular Disease Classification 

Last Edited: 27/4/2020
Author: Shi Jianrong


## Preamble

Cardiovascular disease (CVD) is a class of diseases that involve the heart or blood vessels. It is the leading cause of death in all over the world except in Africa. Some of the common CVD include hypertensive heart disease, stroke, angina, arrhythmia and heart failure.

According to the World Health Organisation (WHO), it takes an estimated 17.9 million lives each year, accounting for 31% of total death worldwide. For patients with existing CVD conditions and symptoms, if proper treatment and intervention are administered, it is estimated that 75% of CVD deaths are preventable. Throughout the years, a large number of research are done in the area of risk factors that affects the chances of getting CVD. Some of the common risk factors identified are genetics, age, gender, tobacco use, alcohol use, physical activities, diet, sleep and stress.

For patients who have not been diagnosed of CVD and who are experiencing symptoms of CVD, doctors will usually evaluate his/her risk based on individual risk factors. Further diagnosis can then be performed ,in the forms of electrocardiogram (ECG), exercise stress tests, echocardiogram, coronary angiography, magnetic resonance imaging (MRI) and computerised tomography (CT), depending on the suspected disease. 

The aim of this project is to predict if a person is suffering from CVD based on a few simple characteristics. The factors that are examined here are age, gender, height, weight, systolic blood pressure, diastolic blood pressure, cholesterol level, glucose level, tabacco use, alcohol use and physical activity. 

The project aims to leverage on machine learning (ML) techniques to effectively classify and detect CVD cases. It is intended to be used as a proof of concept that these factors can be used as a simple evaluation to tell if a potential patient may suffer from CVD. 

## Data Source

The data was obtained from Kaggle, a subsidary under Alphabet Inc. It is an online community for data scientists and ML practitioners. Kaggle allows users to explore the datasets that are uploaded by other users and develop models based on the datasets. This particular dataset is published by Svetlana Ulianova, a data analyst working at TELUS Health, based in Canada. The publisher stated that all the data is collected on the moment of medical examination but the exact source of the data is not known. 

## Methodology

Python will be used here to conduct exploratory data analysis (EDA), as well as to do predictive analysis. Python is chosen due to the ease of use and wide variety of libraries that can be leveraged on. A library is a piece of code that can perform certain actions that has already been written by external sources. This means that as data scientists, we do not have to develop our own code from scratch every time we do data analysis. As ML usually leverage on constant data processing, Python is the ideal general purpose programming language that we can use, allowing us to manipulate and transform data with ease using the wide array of libraries.

Here is a table for some of the libraries that are used.

| Library | Python Library Name  | Purpose |
|:---:|:---:|:---:|
| Scikit-learn | sklearn | Wide variety of ML algorithms, both classification and regression problems |
| Pandas | pandas | Reading and writing of csv file, data manipulation, operations such as merging and joining of files |


## Exploratory Data Analysis